In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import xarray as xr

In [63]:
plastics = xr.open_dataset('../data/plastics.nc',decode_times=False)

In [47]:
plastics

<xarray.Dataset>
Dimensions:  (lat: 192, lon: 288, bin: 6, nvars: 5)
Coordinates:
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 87.17 88.12 89.06 90.0
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
  * bin      (bin) int32 1 2 3 4 5 6
  * nvars    (nvars) int32 1 2 3 4 5
Data variables:
    source   (bin, nvars, lat, lon) float64 ...
    dep      (bin, nvars, lat, lon) float64 ...
    conc     (bin, nvars, lat, lon) float64 ...
    gw       (lat) float64 ...

In [48]:
ocean = plastics['source'][:,0,:,:]

In [49]:
plastics['source'][:,0,:,:]=ocean/1e4

**1st**

In [54]:
import netCDF4 as nc   # Reading NetCDF4 files.
import numpy   as np   # For array operations.

## INPUT FILES
##agricutural
#infile = '../data/plastics.nc'
#indata = nc.Dataset(infile, 'r')

## READ DATA
#6 bins are included, which represent in the following order: 0.3, 2.5, 7, 15, 35 and 70 micrometers in diameter particles
#5 model sources (nvars) are included in the following order:  ocean, pop, dustpop, dust ag, roads, as described in the paper.
bini=plastics.variables['bin'][:]
nvarsi=plastics.variables['nvars'][:]
lati=plastics.variables['lat'][:]
loni=plastics.variables['lon'][:]
sources=plastics.variables['source'][:,:,:,:]

## SPLIT 6 BINS
## Multiple 6.e22 to change unit from kg/m2/s to molecule/cm2/s
## Change the number and unit_string if necessary

#emiss=sources
unit_string="kg/m2/s"
emiss=sources
#unit_string="molecular/cm2/s"
#emiss=6.e22*sources
#print(np.shape(emiss))

nsource = len(nvarsi)
source_string = ["ocean", "pop", "dustpop", "dustag", "roads"]
print('Number of sources:',nsource)

nbin = len(bini)
bin_string =['0p3','2p5','7','15','35','70'] #rename them if necessary
print('Number of bins:',nbin)

## OUTPUT EMISSION DATA PER SOURCE
## ONE FILE WITH ALL BINS
#for j in range(nbin): # use range(4) to only go through the first 4 bins
for j in range(6):
  print("Processing Bin No. ",j+1)
  outfile='./new_ncfile/plastics_bin%s.nc'%(bin_string[j])
  outfile = nc.Dataset(outfile, 'w',format='NETCDF4_CLASSIC')
   ## CREATE dimensions AND variables
  nvars=outfile.createDimension('nvars',nsource)
  lat=outfile.createDimension('lat',len(lati))
  lon=outfile.createDimension('lon',len(loni))
  nvarss=outfile.createVariable('nvars',np.int32,('nvars',))
  lats=outfile.createVariable('lat','f4',('lat',))
  lons=outfile.createVariable('lon','f4',('lon',))
  Ems=outfile.createVariable('emission',np.float32,('nvars','lat','lon'))
  ## WRITE DATA
  lons[:]=loni
  lons.long_name="longitude"
  lons.units="Degree E"
  lats[:]=lati
  lats.long_name="latitude"
  lats.units="Degree N"
  nvarss[:]=nvarsi
  Ems[:,:,:]=emiss[j,:,:]
  Ems.long_name="Microplastics in bin %s"%(bin_string[j])
  Ems.units="%s"%(unit_string)
  outfile.close()

Number of sources: 5
Number of bins: 6
Processing Bin No.  1
Processing Bin No.  2
Processing Bin No.  3
Processing Bin No.  4
Processing Bin No.  5
Processing Bin No.  6


**2nd**

https://github.com/NCAR/musica-tutorial/blob/main/Rewrite_output.ipynb

In [2]:
plastics = xr.open_dataset('/home/m/m300814/python_code/new_ncfile/plastics_bin0p3.nc',decode_times=False)

In [3]:
plastics

<xarray.Dataset>
Dimensions:   (nvars: 5, lat: 192, lon: 288)
Coordinates:
  * nvars     (nvars) int32 1 2 3 4 5
  * lat       (lat) float32 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float32 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
Data variables:
    emission  (nvars, lat, lon) float32 ...